In [ ]:
"""
Code done by,
  Pratik Shetty AI21MTECH12005
  Varshita Sharma AI21MTECH14009
  Sunamdha Harini AI21MTECH14002
  
"""

In [1]:
import nltk
nltk.download('stopwords')

import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence 
from keras.preprocessing import text as tt
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import gensim.downloader as api

[nltk_data] Downloading package stopwords to /home/pratik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
%ls

In [2]:
import pandas as pd
# df=pd.read_excel('/content/drive/My Drive/Imocap_text/processed_tran.xlsx')
df_label=pd.read_excel('scrap_label.xlsx')
# df_label_enc = pd.read_excel('scrap_label_enc.xlsx')
df_text=pd.read_excel('scrap_text.xlsx')

In [3]:
docs=[]
# for text,label in zip(df['text'],df['label']):
#   if(label!=-1):
#     docs.append({'text':text,'label':label})
# print(len(docs))
needed_labels = ['neu', 'sad' , 'hap' , 'ang']
# docs = np.array([])

for sessionid, label in zip(df_label['sessionID'] ,df_label['label']):
#   print(label)
  if label not in needed_labels:
    continue 
  text = df_text[df_text['sessionID']==sessionid]['text']
  if len(text.values) == 0:
    continue
  else:
    text = text.values[0]
  label_enc = needed_labels.index(label)
#   print(label_enc)
  docs.append({'text': text , 'label' : label_enc})
print(len(docs))

2233


In [4]:
import random
random.shuffle(docs)
random.shuffle(docs)
random.shuffle(docs)
total_length=len(docs)
train_length=int(.9*total_length)
train_list=docs[0:train_length]
test_list=docs[train_length:]
print('no of items for train ',len(train_list))
print('no of items for test ',len(test_list))

no of items for train  2009
no of items for test  224


In [19]:
X_train_list = []
y_train_list = []
X_test_list = []
y_test_list = []

for train_l in train_list :
  X_train_list.append(train_l['text'])
  y_train_list.append(train_l['label'])

  
for test_l in test_list :
  X_test_list.append(test_l['text'])
  y_test_list.append(test_l['label'])


In [20]:
X_train = np.array(X_train_list)
y_train = np.array(y_train_list)

X_test = np.array(X_test_list)
y_test = np.array(y_test_list)

In [7]:
import re

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
train_corpus = []
for i in range(0, len(X_train)):
    review = re.sub('[^a-zA-Z]', ' ', X_train[i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    train_corpus.append(review)

In [8]:
import re

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
test_corpus = []
for i in range(0, len(X_test)):
    review = re.sub('[^a-zA-Z]', ' ', X_test[i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    test_corpus.append(review)

In [9]:
# print("Beforeeaning Data")
X_train = np.array(train_corpus)


In [10]:
X_test = np.array(test_corpus)


In [11]:
y_train_enc_nn = np_utils.to_categorical(y_train)
# y_cv_enc_nn = np_utils.to_categorical(y_cv_enc)
y_test_enc_nn = np_utils.to_categorical(y_test)


In [13]:
token = tt.Tokenizer(num_words=None)

max_len = 200
a = X_train.tolist()+ X_test.tolist()
token.fit_on_texts(a)

xtrain_seq = token.texts_to_sequences(X_train)
# xvalid_seq = token.texts_to_sequences(X_cv['clean_utterance'])
xtest_seq = token.texts_to_sequences(X_test)

xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
# xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index

In [ ]:
# embedding_matrix = np.zeros((len(word_index) + 1, 300))
# for word, i in tqdm(word_index.items()):
#     embedding_vector = word_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

In [16]:
embed_dim = 150
lstm_out = 200
max_features = 200
model = Sequential()
model.add(Embedding(len(word_index) + 1, embed_dim,input_length = xtrain_pad.shape[1]))
model.add(LSTM(lstm_out, dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 150)          217950    
                                                                 
 lstm_1 (LSTM)               (None, 200)               280800    
                                                                 
 dense_1 (Dense)             (None, 4)                 804       
                                                                 
Total params: 499,554
Trainable params: 499,554
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

history=model.fit(xtrain_pad, y=y_train_enc_nn, batch_size=25, epochs=300, verbose=1, validation_data=(xtest_pad, y_test_enc_nn),callbacks = [earlystop])

Epoch 1/300
81/81 [==============================] - 40s 500ms/step - loss: 0.3061 - accuracy: 0.8701 - val_loss: 1.5475 - val_accuracy: 0.6295
Epoch 2/300
81/81 [==============================] - 42s 514ms/step - loss: 0.3008 - accuracy: 0.8701 - val_loss: 1.5731 - val_accuracy: 0.6027
Epoch 3/300
81/81 [==============================] - 40s 499ms/step - loss: 0.2908 - accuracy: 0.8736 - val_loss: 1.6153 - val_accuracy: 0.6071
Epoch 4/300
81/81 [==============================] - 40s 493ms/step - loss: 0.2893 - accuracy: 0.8771 - val_loss: 1.5738 - val_accuracy: 0.6116
